# Redis Deep Dive

## Overview

Redis (Remote Dictionary Server) is an open-source, in-memory data structure store used as a database, cache, message broker, and streaming engine. This notebook provides a comprehensive exploration of Redis data structures, Python integration, and caching patterns.

### Topics Covered

| Section | Description |
|---------|-------------|
| **Data Structures** | Strings, Hashes, Lists, Sets, Sorted Sets |
| **Python redis-py** | Connection, operations, pipelining |
| **Caching Patterns** | Cache-aside, Write-through, Write-behind |
| **TTL & Expiration** | Time-to-live strategies, eviction policies |

---

## 1. Redis Architecture Overview

```
┌─────────────────────────────────────────────────────────────────┐
│                        Redis Server                             │
├─────────────────────────────────────────────────────────────────┤
│  ┌─────────────┐  ┌─────────────┐  ┌─────────────────────────┐  │
│  │   Strings   │  │   Hashes    │  │         Lists           │  │
│  │  key:value  │  │ field:value │  │  [elem1, elem2, ...]    │  │
│  └─────────────┘  └─────────────┘  └─────────────────────────┘  │
│  ┌─────────────┐  ┌─────────────┐  ┌─────────────────────────┐  │
│  │    Sets     │  │ Sorted Sets │  │        Streams          │  │
│  │  {members}  │  │ score:member│  │   Time-series data      │  │
│  └─────────────┘  └─────────────┘  └─────────────────────────┘  │
├─────────────────────────────────────────────────────────────────┤
│                    In-Memory Storage                            │
│              (Optional: RDB/AOF Persistence)                    │
└─────────────────────────────────────────────────────────────────┘
```

### Key Characteristics

| Feature | Description |
|---------|-------------|
| **In-Memory** | All data stored in RAM for sub-millisecond latency |
| **Single-Threaded** | Event loop handles commands sequentially (atomic operations) |
| **Persistence** | RDB snapshots and/or AOF (Append Only File) |
| **Replication** | Master-replica architecture for high availability |
| **Clustering** | Horizontal scaling with automatic sharding |

---

## 2. Python redis-py Setup

### Installation

```bash
pip install redis
```

In [ ]:
import redis
from datetime import timedelta
import json
from typing import Optional, Any, Dict, List

# Connection options
# Option 1: Direct connection
r = redis.Redis(
    host='localhost',
    port=6379,
    db=0,
    decode_responses=True  # Automatically decode bytes to strings
)

# Option 2: Connection URL
# r = redis.from_url('redis://localhost:6379/0')

# Option 3: Connection Pool (recommended for production)
pool = redis.ConnectionPool(
    host='localhost',
    port=6379,
    db=0,
    max_connections=10,
    decode_responses=True
)
r_pooled = redis.Redis(connection_pool=pool)

# Test connection
try:
    r.ping()
    print("Connected to Redis successfully!")
except redis.ConnectionError:
    print("Could not connect to Redis. Examples will show expected behavior.")

---

## 3. Redis Data Structures

### 3.1 Strings

The most basic Redis data type. Can store text, serialized JSON, or binary data up to 512MB.

```
┌─────────────────────────────────────────┐
│              STRING                      │
├─────────────────────────────────────────┤
│  Key: "user:1001:name"                  │
│  Value: "John Doe"                      │
├─────────────────────────────────────────┤
│  Key: "counter:visits"                  │
│  Value: "42" (can INCR/DECR)            │
└─────────────────────────────────────────┘
```

In [ ]:
# STRING Operations

class RedisStringOperations:
    """Demonstrates Redis string operations."""
    
    def __init__(self, client: redis.Redis):
        self.r = client
    
    def basic_operations(self):
        """Basic SET/GET operations."""
        # SET - store a value
        self.r.set('user:1001:name', 'John Doe')
        
        # GET - retrieve a value
        name = self.r.get('user:1001:name')
        print(f"Name: {name}")
        
        # SET with expiration (EX = seconds, PX = milliseconds)
        self.r.set('session:abc123', 'user_data', ex=3600)  # 1 hour
        
        # SETEX - shorthand for SET with EX
        self.r.setex('temp:data', 300, 'expires in 5 minutes')
        
        # SETNX - SET if Not eXists (atomic)
        was_set = self.r.setnx('lock:resource', 'holder_id')
        print(f"Lock acquired: {was_set}")
        
        # MSET/MGET - multiple operations
        self.r.mset({
            'user:1001:email': 'john@example.com',
            'user:1001:age': '30'
        })
        values = self.r.mget('user:1001:name', 'user:1001:email', 'user:1001:age')
        print(f"User data: {values}")
    
    def atomic_counters(self):
        """Atomic increment/decrement operations."""
        # INCR/DECR - atomic counters
        self.r.set('counter:visits', 0)
        
        # Increment by 1
        new_val = self.r.incr('counter:visits')
        print(f"After INCR: {new_val}")
        
        # Increment by N
        new_val = self.r.incrby('counter:visits', 10)
        print(f"After INCRBY 10: {new_val}")
        
        # Float increment
        self.r.set('counter:balance', '100.50')
        new_val = self.r.incrbyfloat('counter:balance', 0.75)
        print(f"After INCRBYFLOAT: {new_val}")
    
    def string_manipulation(self):
        """String manipulation operations."""
        self.r.set('message', 'Hello World')
        
        # APPEND
        self.r.append('message', '!')
        print(f"After append: {self.r.get('message')}")
        
        # STRLEN
        length = self.r.strlen('message')
        print(f"Length: {length}")
        
        # GETRANGE (substring)
        substring = self.r.getrange('message', 0, 4)
        print(f"Substring [0:4]: {substring}")
        
        # SETRANGE (overwrite at offset)
        self.r.setrange('message', 6, 'Redis')
        print(f"After SETRANGE: {self.r.get('message')}")


# Example usage (requires running Redis)
# ops = RedisStringOperations(r)
# ops.basic_operations()
# ops.atomic_counters()
# ops.string_manipulation()

### 3.2 Hashes

Redis hashes are maps between string fields and string values, perfect for representing objects.

```
┌─────────────────────────────────────────────────────┐
│                    HASH                              │
├─────────────────────────────────────────────────────┤
│  Key: "user:1001"                                   │
│  ┌────────────┬─────────────────────────────┐       │
│  │   Field    │           Value             │       │
│  ├────────────┼─────────────────────────────┤       │
│  │   name     │       "John Doe"            │       │
│  │   email    │    "john@example.com"       │       │
│  │   age      │          "30"               │       │
│  │   city     │       "New York"            │       │
│  └────────────┴─────────────────────────────┘       │
└─────────────────────────────────────────────────────┘
```

In [ ]:
# HASH Operations

class RedisHashOperations:
    """Demonstrates Redis hash operations."""
    
    def __init__(self, client: redis.Redis):
        self.r = client
    
    def basic_operations(self):
        """Basic hash operations."""
        # HSET - set field(s) in hash
        self.r.hset('user:1001', mapping={
            'name': 'John Doe',
            'email': 'john@example.com',
            'age': '30',
            'city': 'New York'
        })
        
        # HGET - get single field
        name = self.r.hget('user:1001', 'name')
        print(f"Name: {name}")
        
        # HMGET - get multiple fields
        values = self.r.hmget('user:1001', 'name', 'email')
        print(f"Name and email: {values}")
        
        # HGETALL - get all fields and values
        user = self.r.hgetall('user:1001')
        print(f"Full user: {user}")
        
        # HKEYS / HVALS
        fields = self.r.hkeys('user:1001')
        values = self.r.hvals('user:1001')
        print(f"Fields: {fields}, Values: {values}")
    
    def field_operations(self):
        """Field-level operations."""
        # HEXISTS - check if field exists
        exists = self.r.hexists('user:1001', 'email')
        print(f"Email exists: {exists}")
        
        # HDEL - delete field
        self.r.hdel('user:1001', 'city')
        
        # HLEN - number of fields
        count = self.r.hlen('user:1001')
        print(f"Field count: {count}")
        
        # HSETNX - set only if not exists
        was_set = self.r.hsetnx('user:1001', 'phone', '555-1234')
        print(f"Phone was set: {was_set}")
    
    def numeric_operations(self):
        """Numeric field operations."""
        self.r.hset('product:1', mapping={
            'name': 'Widget',
            'price': '19.99',
            'stock': '100'
        })
        
        # HINCRBY - increment integer field
        new_stock = self.r.hincrby('product:1', 'stock', -5)
        print(f"New stock after sale: {new_stock}")
        
        # HINCRBYFLOAT - increment float field
        new_price = self.r.hincrbyfloat('product:1', 'price', 2.50)
        print(f"New price: {new_price}")


# User object pattern with hash
class UserRepository:
    """Repository pattern for user objects using Redis hashes."""
    
    def __init__(self, client: redis.Redis):
        self.r = client
        self.prefix = 'user'
    
    def _key(self, user_id: str) -> str:
        return f"{self.prefix}:{user_id}"
    
    def create(self, user_id: str, data: Dict[str, Any]) -> bool:
        """Create a new user."""
        key = self._key(user_id)
        if self.r.exists(key):
            return False
        self.r.hset(key, mapping=data)
        return True
    
    def get(self, user_id: str) -> Optional[Dict[str, str]]:
        """Get user by ID."""
        data = self.r.hgetall(self._key(user_id))
        return data if data else None
    
    def update(self, user_id: str, updates: Dict[str, Any]) -> bool:
        """Update user fields."""
        key = self._key(user_id)
        if not self.r.exists(key):
            return False
        self.r.hset(key, mapping=updates)
        return True
    
    def delete(self, user_id: str) -> bool:
        """Delete a user."""
        return self.r.delete(self._key(user_id)) > 0

### 3.3 Lists

Redis lists are linked lists of strings, supporting push/pop from both ends. Ideal for queues, stacks, and recent items.

```
┌─────────────────────────────────────────────────────────────────┐
│                          LIST                                   │
├─────────────────────────────────────────────────────────────────┤
│  Key: "queue:notifications"                                    │
│                                                                 │
│  HEAD ◄─── LPUSH                           RPUSH ───► TAIL     │
│    │                                                     │      │
│    ▼                                                     ▼      │
│  ┌─────┐   ┌─────┐   ┌─────┐   ┌─────┐   ┌─────┐              │
│  │msg5 │◄─►│msg4 │◄─►│msg3 │◄─►│msg2 │◄─►│msg1 │              │
│  └─────┘   └─────┘   └─────┘   └─────┘   └─────┘              │
│    │                                                     │      │
│    ▼                                                     ▼      │
│  LPOP ───►                             ◄─── RPOP               │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
# LIST Operations

class RedisListOperations:
    """Demonstrates Redis list operations."""
    
    def __init__(self, client: redis.Redis):
        self.r = client
    
    def push_pop_operations(self):
        """Push and pop operations."""
        # Clear any existing list
        self.r.delete('mylist')
        
        # RPUSH - add to tail (right)
        self.r.rpush('mylist', 'a', 'b', 'c')
        print(f"After RPUSH: {self.r.lrange('mylist', 0, -1)}")
        
        # LPUSH - add to head (left)
        self.r.lpush('mylist', 'z', 'y')
        print(f"After LPUSH: {self.r.lrange('mylist', 0, -1)}")
        
        # RPOP - remove from tail
        item = self.r.rpop('mylist')
        print(f"RPOP returned: {item}")
        
        # LPOP - remove from head
        item = self.r.lpop('mylist')
        print(f"LPOP returned: {item}")
        
        # LLEN - list length
        length = self.r.llen('mylist')
        print(f"List length: {length}")
    
    def range_operations(self):
        """Range and index operations."""
        self.r.delete('numbers')
        self.r.rpush('numbers', *range(10))
        
        # LRANGE - get range of elements (0-indexed, inclusive)
        subset = self.r.lrange('numbers', 2, 5)
        print(f"Elements 2-5: {subset}")
        
        # LINDEX - get element at index
        element = self.r.lindex('numbers', 3)
        print(f"Element at index 3: {element}")
        
        # LSET - set element at index
        self.r.lset('numbers', 0, 'first')
        print(f"After LSET: {self.r.lrange('numbers', 0, 2)}")
        
        # LTRIM - keep only range, remove rest
        self.r.ltrim('numbers', 0, 4)
        print(f"After LTRIM [0:4]: {self.r.lrange('numbers', 0, -1)}")
    
    def blocking_operations(self):
        """Blocking pop operations (for queues)."""
        # BLPOP/BRPOP - blocking pop (waits for element)
        # These are ideal for work queues
        
        # Syntax: BLPOP key1 [key2 ...] timeout
        # result = self.r.blpop('queue:jobs', timeout=5)  # Wait 5 seconds
        # if result:
        #     queue_name, value = result
        #     print(f"Got {value} from {queue_name}")
        
        # BRPOPLPUSH - atomic pop from one list, push to another
        # Great for reliable queue processing
        # self.r.brpoplpush('queue:pending', 'queue:processing', timeout=5)
        pass


# Queue implementation using Lists
class RedisQueue:
    """Simple FIFO queue using Redis lists."""
    
    def __init__(self, client: redis.Redis, name: str):
        self.r = client
        self.key = f"queue:{name}"
    
    def enqueue(self, *items: str) -> int:
        """Add items to the queue."""
        return self.r.rpush(self.key, *items)
    
    def dequeue(self, timeout: int = 0) -> Optional[str]:
        """Remove and return item from queue. Blocks if timeout > 0."""
        if timeout > 0:
            result = self.r.blpop(self.key, timeout=timeout)
            return result[1] if result else None
        return self.r.lpop(self.key)
    
    def peek(self) -> Optional[str]:
        """View next item without removing."""
        return self.r.lindex(self.key, 0)
    
    def size(self) -> int:
        """Get queue size."""
        return self.r.llen(self.key)
    
    def clear(self) -> None:
        """Clear the queue."""
        self.r.delete(self.key)

### 3.4 Sets

Redis sets are unordered collections of unique strings. Perfect for tracking unique items, tags, and set operations.

```
┌─────────────────────────────────────────────────────────────────┐
│                         SETS                                    │
├────────────────────────────┬────────────────────────────────────┤
│  user:1001:interests       │     user:1002:interests            │
│  ┌────────────────────┐    │     ┌────────────────────┐         │
│  │ • python           │    │     │ • python           │         │
│  │ • redis            │    │     │ • javascript       │         │
│  │ • machine_learning │    │     │ • react            │         │
│  │ • docker           │    │     │ • docker           │         │
│  └────────────────────┘    │     └────────────────────┘         │
├────────────────────────────┴────────────────────────────────────┤
│  SINTER (intersection): {python, docker}                        │
│  SUNION (union): {python, redis, machine_learning, docker,      │
│                   javascript, react}                            │
│  SDIFF (1001 - 1002): {redis, machine_learning}                 │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
# SET Operations

class RedisSetOperations:
    """Demonstrates Redis set operations."""
    
    def __init__(self, client: redis.Redis):
        self.r = client
    
    def basic_operations(self):
        """Basic set operations."""
        # SADD - add members
        self.r.sadd('skills:python', 'django', 'flask', 'fastapi', 'redis')
        
        # SMEMBERS - get all members
        members = self.r.smembers('skills:python')
        print(f"Members: {members}")
        
        # SISMEMBER - check membership
        is_member = self.r.sismember('skills:python', 'flask')
        print(f"Flask is member: {is_member}")
        
        # SCARD - cardinality (count)
        count = self.r.scard('skills:python')
        print(f"Skill count: {count}")
        
        # SREM - remove member
        self.r.srem('skills:python', 'flask')
        
        # SPOP - remove and return random member
        random_skill = self.r.spop('skills:python')
        print(f"Random skill removed: {random_skill}")
        
        # SRANDMEMBER - get random member(s) without removing
        random_skills = self.r.srandmember('skills:python', 2)
        print(f"Random skills: {random_skills}")
    
    def set_operations(self):
        """Set theory operations."""
        # Setup
        self.r.delete('set:a', 'set:b')
        self.r.sadd('set:a', 1, 2, 3, 4, 5)
        self.r.sadd('set:b', 4, 5, 6, 7, 8)
        
        # SINTER - intersection
        intersection = self.r.sinter('set:a', 'set:b')
        print(f"Intersection: {intersection}")  # {4, 5}
        
        # SUNION - union
        union = self.r.sunion('set:a', 'set:b')
        print(f"Union: {union}")  # {1, 2, 3, 4, 5, 6, 7, 8}
        
        # SDIFF - difference (in A but not in B)
        diff = self.r.sdiff('set:a', 'set:b')
        print(f"Difference A-B: {diff}")  # {1, 2, 3}
        
        # Store results in new set
        self.r.sinterstore('set:intersection', 'set:a', 'set:b')
        self.r.sunionstore('set:union', 'set:a', 'set:b')
        self.r.sdiffstore('set:diff', 'set:a', 'set:b')


# Use case: Tag system
class TagSystem:
    """Tag system using Redis sets."""
    
    def __init__(self, client: redis.Redis):
        self.r = client
    
    def add_tags(self, item_id: str, *tags: str) -> int:
        """Add tags to an item."""
        # Store tags for item
        self.r.sadd(f"item:{item_id}:tags", *tags)
        
        # Also index items by tag for reverse lookup
        for tag in tags:
            self.r.sadd(f"tag:{tag}:items", item_id)
        return len(tags)
    
    def get_tags(self, item_id: str) -> set:
        """Get all tags for an item."""
        return self.r.smembers(f"item:{item_id}:tags")
    
    def get_items_by_tag(self, tag: str) -> set:
        """Get all items with a specific tag."""
        return self.r.smembers(f"tag:{tag}:items")
    
    def get_items_by_all_tags(self, *tags: str) -> set:
        """Get items that have ALL specified tags."""
        keys = [f"tag:{tag}:items" for tag in tags]
        return self.r.sinter(*keys)
    
    def get_items_by_any_tag(self, *tags: str) -> set:
        """Get items that have ANY of the specified tags."""
        keys = [f"tag:{tag}:items" for tag in tags]
        return self.r.sunion(*keys)

### 3.5 Sorted Sets (ZSets)

Sorted sets combine set uniqueness with ordering by score. Perfect for leaderboards, priority queues, and time-series data.

```
┌─────────────────────────────────────────────────────────────────┐
│                     SORTED SET                                  │
├─────────────────────────────────────────────────────────────────┤
│  Key: "leaderboard:game1"                                      │
│                                                                 │
│  ┌────────┬────────────────┬────────────────────────┐          │
│  │ Score  │    Member      │        Rank            │          │
│  ├────────┼────────────────┼────────────────────────┤          │
│  │  2500  │   player:alice │   0 (highest score)    │          │
│  │  2100  │   player:bob   │   1                    │          │
│  │  1800  │   player:carol │   2                    │          │
│  │  1500  │   player:dave  │   3                    │          │
│  │  1200  │   player:eve   │   4 (lowest score)     │          │
│  └────────┴────────────────┴────────────────────────┘          │
│                                                                 │
│  Operations: ZADD, ZRANGE, ZRANK, ZSCORE, ZINCRBY              │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
# SORTED SET Operations

class RedisSortedSetOperations:
    """Demonstrates Redis sorted set operations."""
    
    def __init__(self, client: redis.Redis):
        self.r = client
    
    def basic_operations(self):
        """Basic sorted set operations."""
        self.r.delete('scores')
        
        # ZADD - add members with scores
        self.r.zadd('scores', {
            'alice': 100,
            'bob': 85,
            'carol': 92,
            'dave': 78
        })
        
        # ZSCORE - get score for member
        score = self.r.zscore('scores', 'alice')
        print(f"Alice's score: {score}")
        
        # ZRANK - get rank (0-based, ascending by score)
        rank = self.r.zrank('scores', 'bob')
        print(f"Bob's rank (ascending): {rank}")
        
        # ZREVRANK - get rank (descending by score)
        rev_rank = self.r.zrevrank('scores', 'bob')
        print(f"Bob's rank (descending): {rev_rank}")
        
        # ZCARD - count members
        count = self.r.zcard('scores')
        print(f"Total members: {count}")
    
    def range_operations(self):
        """Range query operations."""
        # ZRANGE - get by rank range (ascending)
        top3 = self.r.zrange('scores', 0, 2, withscores=True)
        print(f"Bottom 3: {top3}")
        
        # ZREVRANGE - get by rank range (descending)
        top3_desc = self.r.zrevrange('scores', 0, 2, withscores=True)
        print(f"Top 3: {top3_desc}")
        
        # ZRANGEBYSCORE - get by score range
        mid_scorers = self.r.zrangebyscore('scores', 80, 95, withscores=True)
        print(f"Scores 80-95: {mid_scorers}")
        
        # ZCOUNT - count members in score range
        count = self.r.zcount('scores', 80, 95)
        print(f"Count with scores 80-95: {count}")
    
    def modification_operations(self):
        """Score modification operations."""
        # ZINCRBY - increment score
        new_score = self.r.zincrby('scores', 15, 'bob')
        print(f"Bob's new score: {new_score}")
        
        # ZADD with options
        # XX - only update existing, NX - only add new
        # GT - update only if new score > current
        # LT - update only if new score < current
        self.r.zadd('scores', {'alice': 110}, xx=True, gt=True)
        
        # ZREM - remove members
        self.r.zrem('scores', 'dave')
        
        # ZPOPMIN/ZPOPMAX - remove and return min/max
        lowest = self.r.zpopmin('scores', 1)
        print(f"Removed lowest: {lowest}")


# Leaderboard implementation
class Leaderboard:
    """Leaderboard using Redis sorted sets."""
    
    def __init__(self, client: redis.Redis, name: str):
        self.r = client
        self.key = f"leaderboard:{name}"
    
    def add_score(self, player_id: str, score: float) -> float:
        """Set player's score."""
        self.r.zadd(self.key, {player_id: score})
        return score
    
    def increment_score(self, player_id: str, delta: float) -> float:
        """Increment player's score."""
        return self.r.zincrby(self.key, delta, player_id)
    
    def get_rank(self, player_id: str) -> Optional[int]:
        """Get player's rank (1-based, highest score = rank 1)."""
        rank = self.r.zrevrank(self.key, player_id)
        return rank + 1 if rank is not None else None
    
    def get_score(self, player_id: str) -> Optional[float]:
        """Get player's score."""
        return self.r.zscore(self.key, player_id)
    
    def get_top(self, n: int = 10) -> List[tuple]:
        """Get top N players with scores."""
        return self.r.zrevrange(self.key, 0, n - 1, withscores=True)
    
    def get_around_player(self, player_id: str, n: int = 5) -> List[tuple]:
        """Get players around a specific player."""
        rank = self.r.zrevrank(self.key, player_id)
        if rank is None:
            return []
        start = max(0, rank - n // 2)
        return self.r.zrevrange(self.key, start, start + n - 1, withscores=True)
    
    def total_players(self) -> int:
        """Get total number of players."""
        return self.r.zcard(self.key)

---

## 4. Caching Patterns

### 4.1 Cache-Aside (Lazy Loading)

The application checks the cache first. On cache miss, it loads from the database and populates the cache.

```
┌─────────────────────────────────────────────────────────────────┐
│                    CACHE-ASIDE PATTERN                          │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  ┌──────────┐    1. Check Cache     ┌───────────┐              │
│  │          │ ─────────────────────►│           │              │
│  │   App    │    2. Cache Miss      │   Redis   │              │
│  │          │ ◄─────────────────────│  (Cache)  │              │
│  └────┬─────┘                       └───────────┘              │
│       │                                    ▲                    │
│       │ 3. Query DB                        │                    │
│       ▼                                    │                    │
│  ┌──────────┐                              │                    │
│  │          │                              │                    │
│  │ Database │       4. Update Cache ───────┘                    │
│  │          │                                                   │
│  └──────────┘                                                   │
│                                                                 │
│  Pros: Only requested data cached, cache failures non-fatal     │
│  Cons: Cache miss penalty, potential stale data                │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Cache-Aside Pattern Implementation

from typing import Callable, TypeVar
from functools import wraps
import hashlib

T = TypeVar('T')


class CacheAside:
    """Cache-aside pattern implementation."""
    
    def __init__(self, client: redis.Redis, default_ttl: int = 3600):
        self.r = client
        self.default_ttl = default_ttl
    
    def get_or_load(
        self,
        key: str,
        loader: Callable[[], T],
        ttl: Optional[int] = None,
        serializer: Callable[[T], str] = json.dumps,
        deserializer: Callable[[str], T] = json.loads
    ) -> T:
        """Get from cache or load from source."""
        # Try cache first
        cached = self.r.get(key)
        if cached is not None:
            return deserializer(cached)
        
        # Cache miss - load from source
        value = loader()
        
        # Store in cache
        self.r.setex(
            key,
            ttl or self.default_ttl,
            serializer(value)
        )
        
        return value
    
    def invalidate(self, key: str) -> bool:
        """Invalidate cache entry."""
        return self.r.delete(key) > 0
    
    def invalidate_pattern(self, pattern: str) -> int:
        """Invalidate all keys matching pattern."""
        keys = self.r.keys(pattern)
        if keys:
            return self.r.delete(*keys)
        return 0


def cache_aside(cache: CacheAside, key_prefix: str, ttl: int = 3600):
    """Decorator for cache-aside pattern."""
    def decorator(func: Callable) -> Callable:
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Generate cache key from function args
            key_parts = [key_prefix, func.__name__]
            key_parts.extend(str(arg) for arg in args)
            key_parts.extend(f"{k}:{v}" for k, v in sorted(kwargs.items()))
            cache_key = ":".join(key_parts)
            
            return cache.get_or_load(
                cache_key,
                lambda: func(*args, **kwargs),
                ttl=ttl
            )
        
        wrapper.invalidate = lambda *args, **kwargs: cache.invalidate(
            ":".join([key_prefix, func.__name__] + 
                     [str(arg) for arg in args] +
                     [f"{k}:{v}" for k, v in sorted(kwargs.items())])
        )
        return wrapper
    return decorator


# Usage example
class UserService:
    """Example service using cache-aside pattern."""
    
    def __init__(self, redis_client: redis.Redis):
        self.cache = CacheAside(redis_client)
        # Simulated database
        self.db = {
            '1': {'id': '1', 'name': 'Alice', 'email': 'alice@example.com'},
            '2': {'id': '2', 'name': 'Bob', 'email': 'bob@example.com'},
        }
    
    def get_user(self, user_id: str) -> Optional[dict]:
        """Get user with cache-aside pattern."""
        cache_key = f"user:{user_id}"
        
        return self.cache.get_or_load(
            cache_key,
            loader=lambda: self.db.get(user_id),
            ttl=3600
        )
    
    def update_user(self, user_id: str, data: dict) -> bool:
        """Update user and invalidate cache."""
        if user_id not in self.db:
            return False
        
        # Update database
        self.db[user_id].update(data)
        
        # Invalidate cache
        self.cache.invalidate(f"user:{user_id}")
        
        return True

### 4.2 Write-Through Cache

Writes go to both cache and database synchronously. Ensures cache is always consistent.

```
┌─────────────────────────────────────────────────────────────────┐
│                   WRITE-THROUGH PATTERN                         │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  ┌──────────┐    1. Write Request    ┌───────────┐             │
│  │          │ ──────────────────────►│           │             │
│  │   App    │                        │   Redis   │             │
│  │          │                        │  (Cache)  │             │
│  └──────────┘                        └─────┬─────┘             │
│                                            │                    │
│                              2. Write to DB (sync)              │
│                                            │                    │
│                                            ▼                    │
│                                     ┌───────────┐              │
│                                     │           │              │
│                                     │ Database  │              │
│                                     │           │              │
│                                     └───────────┘              │
│                                                                 │
│  Pros: Cache always consistent, simple reads                    │
│  Cons: Write latency, cache may hold unused data               │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Write-Through Pattern Implementation

class WriteThroughCache:
    """Write-through cache pattern implementation."""
    
    def __init__(self, redis_client: redis.Redis, default_ttl: int = 3600):
        self.r = redis_client
        self.default_ttl = default_ttl
    
    def write(
        self,
        key: str,
        value: Any,
        db_writer: Callable[[str, Any], bool],
        ttl: Optional[int] = None,
        serializer: Callable[[Any], str] = json.dumps
    ) -> bool:
        """Write to cache and database synchronously."""
        # Write to cache first
        self.r.setex(
            key,
            ttl or self.default_ttl,
            serializer(value)
        )
        
        # Write to database
        try:
            success = db_writer(key, value)
            if not success:
                # Rollback cache on DB failure
                self.r.delete(key)
                return False
            return True
        except Exception as e:
            # Rollback cache on exception
            self.r.delete(key)
            raise
    
    def read(
        self,
        key: str,
        deserializer: Callable[[str], Any] = json.loads
    ) -> Optional[Any]:
        """Read from cache (data should always be there for writes)."""
        cached = self.r.get(key)
        if cached is not None:
            return deserializer(cached)
        return None


class ProductService:
    """Example service using write-through pattern."""
    
    def __init__(self, redis_client: redis.Redis):
        self.cache = WriteThroughCache(redis_client)
        # Simulated database
        self.db = {}
    
    def _db_write(self, key: str, value: dict) -> bool:
        """Simulated database write."""
        product_id = key.split(':')[1]
        self.db[product_id] = value
        return True
    
    def create_product(self, product_id: str, data: dict) -> bool:
        """Create product with write-through caching."""
        cache_key = f"product:{product_id}"
        return self.cache.write(
            cache_key,
            data,
            self._db_write,
            ttl=7200  # 2 hours
        )
    
    def get_product(self, product_id: str) -> Optional[dict]:
        """Get product from cache."""
        return self.cache.read(f"product:{product_id}")

### 4.3 Write-Behind (Write-Back) Cache

Writes go to cache immediately, database updates happen asynchronously.

```
┌─────────────────────────────────────────────────────────────────┐
│                   WRITE-BEHIND PATTERN                          │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  ┌──────────┐    1. Write Request    ┌───────────┐             │
│  │          │ ──────────────────────►│   Redis   │             │
│  │   App    │    2. ACK (immediate)  │  (Cache)  │             │
│  │          │ ◄──────────────────────│           │             │
│  └──────────┘                        └─────┬─────┘             │
│                                            │                    │
│                              3. Queue for async write           │
│                                            │                    │
│                                            ▼                    │
│                                     ┌───────────┐              │
│                          4. Batch   │  Writer   │              │
│                             Write   │  Process  │              │
│                                     └─────┬─────┘              │
│                                           │                     │
│                                           ▼                     │
│                                     ┌───────────┐              │
│                                     │ Database  │              │
│                                     └───────────┘              │
│                                                                 │
│  Pros: Very low write latency, batch optimization              │
│  Cons: Risk of data loss, eventual consistency                 │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Write-Behind Pattern Implementation

import threading
import time
from queue import Queue


class WriteBehindCache:
    """Write-behind cache pattern with async database writes."""
    
    def __init__(
        self,
        redis_client: redis.Redis,
        db_writer: Callable[[str, Any], bool],
        batch_size: int = 100,
        flush_interval: float = 5.0,
        default_ttl: int = 3600
    ):
        self.r = redis_client
        self.db_writer = db_writer
        self.batch_size = batch_size
        self.flush_interval = flush_interval
        self.default_ttl = default_ttl
        
        # Write queue using Redis list for durability
        self.queue_key = "writebehind:queue"
        
        # Start background writer thread
        self._running = True
        self._writer_thread = threading.Thread(target=self._background_writer)
        self._writer_thread.daemon = True
        self._writer_thread.start()
    
    def write(
        self,
        key: str,
        value: Any,
        ttl: Optional[int] = None
    ) -> bool:
        """Write to cache and queue for async DB write."""
        serialized = json.dumps(value)
        
        # Write to cache
        self.r.setex(key, ttl or self.default_ttl, serialized)
        
        # Queue for database write
        write_item = json.dumps({'key': key, 'value': value})
        self.r.rpush(self.queue_key, write_item)
        
        return True
    
    def read(self, key: str) -> Optional[Any]:
        """Read from cache."""
        cached = self.r.get(key)
        if cached:
            return json.loads(cached)
        return None
    
    def _background_writer(self):
        """Background thread that writes to database."""
        last_flush = time.time()
        batch = []
        
        while self._running:
            # Try to get item from queue (blocking with timeout)
            item = self.r.lpop(self.queue_key)
            
            if item:
                batch.append(json.loads(item))
            
            # Flush if batch is full or interval elapsed
            should_flush = (
                len(batch) >= self.batch_size or
                (batch and time.time() - last_flush >= self.flush_interval)
            )
            
            if should_flush and batch:
                self._flush_batch(batch)
                batch = []
                last_flush = time.time()
            
            # Small sleep to prevent tight loop
            if not item:
                time.sleep(0.1)
    
    def _flush_batch(self, batch: List[dict]):
        """Write batch to database."""
        for item in batch:
            try:
                self.db_writer(item['key'], item['value'])
            except Exception as e:
                # Log error and potentially re-queue
                print(f"Error writing to DB: {e}")
    
    def stop(self):
        """Stop the background writer."""
        self._running = False
        self._writer_thread.join(timeout=10)

### 4.4 Read-Through Cache

The cache itself is responsible for loading data on cache miss.

In [ ]:
# Read-Through Pattern Implementation

class ReadThroughCache:
    """Read-through cache - cache is responsible for loading data."""
    
    def __init__(
        self,
        redis_client: redis.Redis,
        data_loader: Callable[[str], Optional[Any]],
        default_ttl: int = 3600
    ):
        self.r = redis_client
        self.data_loader = data_loader
        self.default_ttl = default_ttl
    
    def get(self, key: str, ttl: Optional[int] = None) -> Optional[Any]:
        """Get data - loads from source if not cached."""
        # Check cache
        cached = self.r.get(key)
        if cached is not None:
            return json.loads(cached)
        
        # Load from data source
        data = self.data_loader(key)
        if data is not None:
            # Store in cache
            self.r.setex(
                key,
                ttl or self.default_ttl,
                json.dumps(data)
            )
        
        return data
    
    def invalidate(self, key: str) -> bool:
        """Invalidate cache entry."""
        return self.r.delete(key) > 0
    
    def refresh(self, key: str, ttl: Optional[int] = None) -> Optional[Any]:
        """Force refresh from data source."""
        self.invalidate(key)
        return self.get(key, ttl)

---

## 5. TTL and Expiration Strategies

### 5.1 TTL Basics

```
┌─────────────────────────────────────────────────────────────────┐
│                    TTL LIFECYCLE                                │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  SET key value EX 300    (TTL = 300 seconds)                   │
│                                                                 │
│  Time: 0s ────────────► 150s ────────────► 300s                │
│         │                 │                  │                  │
│         ▼                 ▼                  ▼                  │
│    [Key Created]   [TTL = 150s]      [Key Expires]             │
│                                                                 │
│  Commands:                                                      │
│  • TTL key       → Returns remaining seconds                   │
│  • PTTL key      → Returns remaining milliseconds              │
│  • EXPIRE key N  → Set new TTL (seconds)                       │
│  • PERSIST key   → Remove TTL (make permanent)                 │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
# TTL Operations

class TTLManager:
    """Manage TTL and expiration for Redis keys."""
    
    def __init__(self, client: redis.Redis):
        self.r = client
    
    def set_with_ttl(self, key: str, value: str, ttl_seconds: int) -> bool:
        """Set a key with TTL."""
        return self.r.setex(key, ttl_seconds, value)
    
    def get_ttl(self, key: str) -> int:
        """Get remaining TTL in seconds.
        Returns: -2 if key doesn't exist, -1 if no TTL, else seconds.
        """
        return self.r.ttl(key)
    
    def get_pttl(self, key: str) -> int:
        """Get remaining TTL in milliseconds."""
        return self.r.pttl(key)
    
    def extend_ttl(self, key: str, additional_seconds: int) -> bool:
        """Extend TTL by additional seconds."""
        current_ttl = self.r.ttl(key)
        if current_ttl > 0:
            return self.r.expire(key, current_ttl + additional_seconds)
        return False
    
    def set_expiration(self, key: str, ttl_seconds: int) -> bool:
        """Set new TTL on existing key."""
        return self.r.expire(key, ttl_seconds)
    
    def set_expiration_at(self, key: str, timestamp: int) -> bool:
        """Set expiration at specific Unix timestamp."""
        return self.r.expireat(key, timestamp)
    
    def remove_expiration(self, key: str) -> bool:
        """Remove TTL, making key permanent."""
        return self.r.persist(key)
    
    def touch_refresh_ttl(self, key: str, new_ttl: int) -> bool:
        """Refresh TTL on access (sliding expiration)."""
        if self.r.exists(key):
            return self.r.expire(key, new_ttl)
        return False

### 5.2 Expiration Strategies

| Strategy | Description | Use Case |
|----------|-------------|----------|
| **Fixed TTL** | Same TTL for all entries | Session data, rate limits |
| **Sliding TTL** | Refresh TTL on access | Active sessions, hot data |
| **Adaptive TTL** | TTL based on access frequency | Intelligent caching |
| **Time-based** | Expire at specific time | Daily resets, scheduled expiry |

In [ ]:
# Advanced Expiration Strategies

import time
from datetime import datetime, timezone


class ExpirationStrategies:
    """Different TTL/expiration strategies."""
    
    def __init__(self, client: redis.Redis):
        self.r = client
    
    # 1. Fixed TTL
    def set_fixed_ttl(self, key: str, value: str, ttl: int = 3600) -> bool:
        """Set with fixed TTL."""
        return self.r.setex(key, ttl, value)
    
    # 2. Sliding TTL (refresh on access)
    def get_with_sliding_ttl(self, key: str, ttl: int = 3600) -> Optional[str]:
        """Get value and refresh TTL."""
        value = self.r.get(key)
        if value is not None:
            self.r.expire(key, ttl)  # Refresh TTL on each access
        return value
    
    # 3. Adaptive TTL based on access frequency
    def set_adaptive_ttl(
        self,
        key: str,
        value: str,
        base_ttl: int = 3600,
        max_ttl: int = 86400
    ) -> bool:
        """Set with adaptive TTL - frequently accessed items get longer TTL."""
        access_key = f"{key}:access_count"
        
        # Increment access count
        access_count = self.r.incr(access_key)
        self.r.expire(access_key, max_ttl)  # Track access count
        
        # Calculate TTL based on access frequency
        # More accesses = longer TTL (up to max)
        ttl = min(base_ttl * (1 + access_count // 10), max_ttl)
        
        return self.r.setex(key, ttl, value)
    
    # 4. Time-based expiration (expire at specific time)
    def set_expire_at_midnight(self, key: str, value: str) -> bool:
        """Set key to expire at next midnight UTC."""
        now = datetime.now(timezone.utc)
        next_midnight = now.replace(
            hour=0, minute=0, second=0, microsecond=0
        )
        if next_midnight <= now:
            next_midnight = next_midnight.replace(day=now.day + 1)
        
        timestamp = int(next_midnight.timestamp())
        
        pipe = self.r.pipeline()
        pipe.set(key, value)
        pipe.expireat(key, timestamp)
        pipe.execute()
        return True
    
    # 5. Probabilistic early expiration (prevent stampede)
    def get_with_early_expiration(
        self,
        key: str,
        loader: Callable[[], str],
        ttl: int = 3600,
        beta: float = 1.0
    ) -> str:
        """XFetch algorithm - probabilistic early recomputation.
        
        Prevents cache stampede by having some requests refresh early.
        """
        import random
        import math
        
        cached = self.r.get(key)
        remaining_ttl = self.r.ttl(key)
        
        if cached is None or remaining_ttl <= 0:
            # Cache miss - definitely need to refresh
            value = loader()
            self.r.setex(key, ttl, value)
            return value
        
        # XFetch: probabilistically refresh before expiration
        # delta = time since cache was set = ttl - remaining_ttl
        delta = ttl - remaining_ttl
        
        # Probability of refresh increases as we approach expiration
        # -delta * beta * log(random()) gives probability distribution
        if delta > 0:
            xfetch_value = delta * beta * -math.log(random.random())
            if xfetch_value >= remaining_ttl:
                # Early refresh
                value = loader()
                self.r.setex(key, ttl, value)
                return value
        
        return cached

### 5.3 Redis Eviction Policies

When Redis reaches `maxmemory`, it uses eviction policies to make room:

| Policy | Description |
|--------|-------------|
| `noeviction` | Return error on writes when memory is full |
| `allkeys-lru` | Evict least recently used keys |
| `volatile-lru` | Evict LRU among keys with TTL set |
| `allkeys-lfu` | Evict least frequently used keys |
| `volatile-lfu` | Evict LFU among keys with TTL set |
| `allkeys-random` | Evict random keys |
| `volatile-random` | Evict random keys with TTL set |
| `volatile-ttl` | Evict keys with shortest TTL |

In [ ]:
# Redis Configuration Commands

def check_eviction_policy(r: redis.Redis) -> dict:
    """Check current memory and eviction configuration."""
    info = r.info('memory')
    config = r.config_get('maxmemory-policy')
    
    return {
        'used_memory_human': info.get('used_memory_human'),
        'maxmemory_human': info.get('maxmemory_human'),
        'eviction_policy': config.get('maxmemory-policy'),
        'evicted_keys': info.get('evicted_keys')
    }


# Example: Configure eviction policy (requires admin privileges)
# r.config_set('maxmemory', '100mb')
# r.config_set('maxmemory-policy', 'allkeys-lru')

---

## 6. Pipelining and Transactions

### 6.1 Pipelining

Send multiple commands in a single network round-trip for better performance.

In [ ]:
# Pipelining for batch operations

def pipeline_example(r: redis.Redis):
    """Demonstrate pipelining for batch operations."""
    
    # Without pipeline: N round-trips
    # for i in range(100):
    #     r.set(f'key:{i}', f'value:{i}')
    
    # With pipeline: 1 round-trip
    pipe = r.pipeline()
    for i in range(100):
        pipe.set(f'key:{i}', f'value:{i}')
    results = pipe.execute()  # Execute all commands
    
    print(f"Set {len(results)} keys")
    
    # Pipeline with mixed commands
    pipe = r.pipeline()
    pipe.set('counter', 0)
    pipe.incr('counter')
    pipe.incr('counter')
    pipe.get('counter')
    results = pipe.execute()
    print(f"Results: {results}")  # [True, 1, 2, '2']


def transaction_example(r: redis.Redis):
    """Demonstrate transactions (MULTI/EXEC)."""
    
    # Transaction: all commands execute atomically
    pipe = r.pipeline(transaction=True)  # Default is True
    
    pipe.multi()  # Start transaction (implicit with pipeline)
    pipe.set('account:1:balance', 1000)
    pipe.set('account:2:balance', 1000)
    pipe.decrby('account:1:balance', 100)
    pipe.incrby('account:2:balance', 100)
    results = pipe.execute()  # EXEC
    
    print(f"Transaction results: {results}")


def optimistic_locking_example(r: redis.Redis):
    """Demonstrate WATCH for optimistic locking."""
    
    def transfer_money(from_account: str, to_account: str, amount: int) -> bool:
        """Transfer money with optimistic locking."""
        from_key = f"account:{from_account}:balance"
        to_key = f"account:{to_account}:balance"
        
        with r.pipeline() as pipe:
            while True:
                try:
                    # Watch for changes
                    pipe.watch(from_key, to_key)
                    
                    # Get current balances
                    from_balance = int(pipe.get(from_key) or 0)
                    
                    if from_balance < amount:
                        pipe.unwatch()
                        return False  # Insufficient funds
                    
                    # Start transaction
                    pipe.multi()
                    pipe.decrby(from_key, amount)
                    pipe.incrby(to_key, amount)
                    pipe.execute()
                    return True
                    
                except redis.WatchError:
                    # Key was modified, retry
                    continue
    
    return transfer_money

---

## 7. Best Practices Summary

### Key Naming Conventions

```
object-type:id:field

Examples:
• user:1001:profile
• session:abc123
• cache:products:category:electronics
• queue:email:pending
• rate-limit:api:user:1001
```

### Performance Tips

| Tip | Description |
|-----|-------------|
| Use connection pools | Avoid connection overhead |
| Pipeline commands | Reduce network round-trips |
| Set appropriate TTLs | Prevent memory bloat |
| Use SCAN, not KEYS | KEYS blocks the server |
| Choose right data structure | Hashes vs Strings for objects |
| Avoid large keys/values | Keep values under 100KB |

### Common Pitfalls

| Pitfall | Solution |
|---------|----------|
| Cache stampede | Use locking or probabilistic early refresh |
| Hot keys | Distribute across replicas or use local cache |
| Memory exhaustion | Set maxmemory and eviction policy |
| Blocking operations | Use timeouts, avoid KEYS in production |

---

## 8. Quick Reference

### Data Structure Selection Guide

| Use Case | Data Structure | Key Commands |
|----------|---------------|---------------|
| Simple key-value | String | GET, SET, INCR |
| Object with fields | Hash | HGET, HSET, HGETALL |
| Queue/Stack | List | LPUSH, RPOP, BLPOP |
| Unique items, tags | Set | SADD, SMEMBERS, SINTER |
| Ranking, scores | Sorted Set | ZADD, ZRANGE, ZRANK |
| Time-series, logs | Stream | XADD, XREAD, XRANGE |

### Essential Commands Cheatsheet

```python
# Connection
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

# Strings
r.set('key', 'value')           r.get('key')
r.setex('key', 3600, 'value')   r.incr('counter')

# Hashes
r.hset('hash', 'field', 'val')  r.hget('hash', 'field')
r.hset('hash', mapping={...})   r.hgetall('hash')

# Lists
r.rpush('list', 'item')         r.lpop('list')
r.lrange('list', 0, -1)         r.blpop('list', timeout=5)

# Sets
r.sadd('set', 'member')         r.smembers('set')
r.sinter('set1', 'set2')        r.sismember('set', 'member')

# Sorted Sets
r.zadd('zset', {'m': 1.0})      r.zrange('zset', 0, -1)
r.zscore('zset', 'member')      r.zrank('zset', 'member')

# TTL
r.expire('key', 3600)           r.ttl('key')
r.persist('key')                r.expireat('key', timestamp)

# Pipeline
pipe = r.pipeline()
pipe.set('k1', 'v1')
pipe.get('k1')
results = pipe.execute()
```